<h1>ECON 140R Class 13</h1>

In this notebook we're examining a special extract of the 1980 Census to replicate [Angrist and Krueger (1994)](https://www-jstor-org.libproxy.berkeley.edu/stable/2535121), who research "Why Do World War II Veterans Earn More than Nonveterans?" 

Learning objectives:

1. More experience with OLS
2. Setting us up for IV
3. Box plots

In [ ]:
library(tidyverse)
library(haven)
library(ggplot2)

It turns out that this handy command stops __R__ from defaulting to scientific notation. 

In [ ]:
options(scipen=999)

<hr>

This is an extract downloaded from IPUMS of the 1980 Census public-use microsample, a 5% flat cut of the entire Census. This particular sample contains men only, who were born in 1925, 1926, 1927, and 1928. The data also contain their quarter of birth, whether they served in WWII `wwii`, and their wage and salary income `incwage`, and several other characteristics.

The 1980 Census was unique in asking about month of birth on the [short form](https://www.census.gov/history/pdf/1980_short_questionnaire.pdf), which everyone answered. The public-use microdata sample condensed this into quarter of birth, but `birthqtr` is still fairly unique across Census products. Some other datasets measure month of birth, and restricted-use datasets may also supply even day of birth.

You can download these data yourself. But be advised the the full dataset contains over 11 million records, or 5% of the nation's roughly 227 million people in 1980. Even with just 36 variables selected, the extract is over a gigabyte in size and is too large for datahub.

<hr>

Let's load up and look at the data.

In [ ]:
data_c80_regsample = read_dta("data_c80_regsample_1.dta")
head(data_c80_regsample)

Here is a baseline regression of a useful $Y$ variable, log pre-tax wage and salary income, which is [described at IPUMS here](https://usa.ipums.org/usa-action/variables/INCWAGE#description_section):

$$
\ln Y_i = \alpha + \beta^{w} \ wwii_i + B \ controls_i + \epsilon_i
$$

where we are controlling for 0/1 WWII service; year of birth; being white (Black, Hispanic, and other men are the baseline omitted category); being married in 1980; a 0/1 indicator of living and working in a standard metropolitan statistical area (SMSA); years of education; a 0/1 indicator of a disability that limits or prevents work; and 49 indicators for 48 lower states (AK and HI are dropped) plus DC.

We'll run this regression and examine what we find for $\beta^w$. Let's follow what [Angrist and Krueger (1994)](https://www-jstor-org.libproxy.berkeley.edu/stable/2535121) do in the left side of Table 4, marked "OLS," which looks a lot like the left-hand side of Table 2.2 in <i>Mastering Metrics</i> Chapter 2. In both, the authors start with a simple model and the add some covariates that might have (and did) inject omitted variable bias. Here's what we'll do:

1. $\ln Y_i = \alpha + \beta^{w} \ wwii_i + \sum \beta^{by} \ birthyear_i  + \epsilon_i$

2. $\ln Y_i = \alpha + \beta^{w} \ wwii_i + \sum \beta^{by} \ birthyear_i  + \beta^wnh \ white_i + \beta^m \ married_i + \sum \beta^{s}\ state_i + \beta^u \ SMSA_i + \epsilon_i$

2. $\ln Y_i = \alpha + \beta^{w} \ wwii_i + \sum \beta^{by} \ birthyear_i  + \beta^wnh \ white_i + \beta^m \ married_i + \sum \beta^{s}\ state_i + \beta^u \ SMSA_i + \beta^e \ educ_i + \beta^d \ disability_i + \epsilon_i$

In [ ]:
c80_reg1 <- lm(logincwage ~ wwii + factor(birthyr), 
               data = data_c80_regsample)
summary(c80_reg1)

In [ ]:
c80_reg2 <- lm(logincwage ~ wwii + factor(birthyr) + white + married 
               + factor(statefip) + smsa, 
               data = data_c80_regsample)
summary(c80_reg2)

In [ ]:
c80_reg3 <- lm(logincwage ~ wwii + factor(birthyr) + white + married 
               + factor(statefip) + smsa 
               + edyrs + disability, 
               data = data_c80_regsample)
summary(c80_reg3)

What do you see? Discuss. If we are concerned about the impact of WWII service on wages, do you think those extra right-hand-side variables are more like omitted variables, or more like bad controls?

Unfortunately `ggplot2` and `haven` apparently don't always play well together. When I first tried running the boxplot code further below, I got weird error messages and nothing. Tips for when (not if) __R__ squawks and hoses you:

* Deep breath. Sigh. Laugh. Post angrily to Twitter?
* Copy the error message and paste into Google
* Look for a Stack Exchange [stackoverflow.com](https://stackoverflow.com) post
* Profit


The code below apparently cleans up these indicator variables for use in `ggplot2`. Don't ask me why, because I don't know! The variable `notwwii` is obvious and it was useful because it switches the order. The variable `srgrp` is useful because of data labels for its values that I inserted in Stata. It measures 3 subgroups shown in an JASA paper by [Small and Rosenbaum (2008)](https://www-tandfonline-com.libproxy.berkeley.edu/doi/abs/10.1198/016214507000001247). 
1. Born in 1924 Q3 or Q4
2. Born in 1926 Q3 or Q4
3. Born in 1928 Q3 or Q4

Note that our extract includes men born in 1925 through 1928, in concordance with the sample drawn by [Angrist and Krueger (1994)](https://www-jstor-org.libproxy.berkeley.edu/stable/2535121). So the men born in 1924 aren't present and won't show up in the box plots. (Which is OK, they look a lot like the men born in 1926.)

In [ ]:
data_c80_regsample$notwwii <- haven::as_factor(data_c80_regsample$notwwii)
data_c80_regsample$srgrp   <- haven::as_factor(data_c80_regsample$srgrp)

Now that we have that glitch figured out, here is a replication of Figure 1 in [Small and Rosenbaum (2008)](https://www-tandfonline-com.libproxy.berkeley.edu/doi/abs/10.1198/016214507000001247), a very nice review of what Angrist and Krueger had done. <b>Box plots</b> are more common in other disciplines than in economics, and they depict visually what economists and sociologists might instead place in a table: means, centiles, outliers, and so on. In its box plots, __R__'s `ggplot2` shows us the 25th, 50th, and 75th percentiles, and two "whiskers." The whiskers extend by default to 1.5 times the interquartile range, which is the distance between 25th and 75th percentiles. [Details here](https://www.rdocumentation.org/packages/ggplot2/versions/3.3.6/topics/geom_boxplot).

In [ ]:
ggplot(subset(data_c80_regsample, birthyr == 1926 & (birthqtr == 3 | birthqtr == 4)), 
              aes(notwwii, incwage)) + geom_boxplot()

Describe what you see. Who earns more among these men born in Q3 or Q4 of 1926, WWII veterans or nonveterans?

<hr>

Now consider this comparison. Let's look at two groups separated by birth year:

1. Men born in Q3 or Q4 of 1926
2. Men born in Q3 or Q4 of 1928

I've coded this using the categorical variable `srgrp`, because it helpfully places the value labels along the bottom of the plot. The labels shown the percentages of these two groups that are veterans of WWII. For the 1926 group and many birth years before, the WWII veteran share is around 75%. For the 1928 group, it is about 25%. 

In [ ]:
ggplot(subset(data_c80_regsample, srgrp != "NA"), 
       aes(srgrp, incwage)) + geom_boxplot()

Describe what you see. Despite the large difference in the shares of these groups that are veterans of WWII, are there large differences in earnings?

For kicks, I've written the same code without `srgrp` except in the label, to make clearer what we're looking at:

In [ ]:
ggplot(subset(data_c80_regsample, (birthyr == 1926 | birthyr == 1928) & 
              (birthqtr == 3 | birthqtr == 4)), 
       aes(srgrp, incwage)) + geom_boxplot()

Where to next? Why do we see such large differences in the WWII veteran share but not in earnings across these two groups? What do birth year and quarters have to do with it?

<i>As warfare and killing rage again in Europe in 2022, let's also take a moment to recognize the great human costs and sacrifices associated with armed conflict and open warfare, and the tragedy of nuclear war.</i>

<hr>

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>